In [ ]:
import torch
import clip
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import CocoCaptions
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import torch.nn as nn
import torch.optim

In [13]:
# Устройство для выполнения вычислений
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузим модель CLIP и инициализируем tokenizer
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
model = model.float().train()
tokenizer = clip.tokenize

In [14]:
# Оптимизатор и функция потерь
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6, weight_decay=0.01) 

In [15]:
# Добавлен Dropout в модель
dropout_rate = 0.2  # Можно настроить этот параметр
model.visual.dropout = nn.Dropout(dropout_rate)  # Применяется к визуальной ветви модели
model.transformer.dropout = nn.Dropout(dropout_rate)  # Применяется к текстовой ветви модели


In [16]:
# Аугментация и нормализация
transform = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # Преобразование в тензор
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]),
    transforms.Lambda(lambda x: torch.clamp(x, min=0, max=1)),
])

In [17]:
class CocoDataset(Dataset):
    def __init__(self, root, annFile, transform=None, max_samples=None):
        super().__init__()
        self.coco = CocoCaptions(root=root, annFile=annFile)
        self.transform = transform  # Сохраняем transform
        self.max_samples = max_samples  # Максимальное количество данных для загрузки

        # Если указано max_samples, ограничиваем количество данных
        if self.max_samples is not None:
            self.indices = list(range(min(len(self.coco), self.max_samples)))
        else:
            self.indices = list(range(len(self.coco)))  # Иначе используем весь датасет

    def __len__(self):
        return len(self.indices)  # Возвращаем количество выбранных данных
        
    def __getitem__(self, idx):
        # Используем индекс из self.indices
        actual_idx = self.indices[idx]
        image, captions = self.coco[actual_idx]
        caption = captions[0]  # Используем первое описание
        if self.transform is not None:
            image = self.transform(image)  # Применяем transform
        return image, caption

In [18]:
# Загрузчик данных
def collate_fn(batch):
    images, captions = [], []
    for item in batch:
        images.append(item[0].unsqueeze(0))  # Добавляем изображение
        captions.append(item[1])  # Добавляем текст
    images = torch.cat(images, dim=0)  # Объединяем изображения в батч
    captions = tokenizer(captions, truncate=True).to(device)  # Токенизируем текст
    return images, captions

In [20]:
# Создание DataLoader
test_dataset = CocoDataset(
    root="coco2017/train2017",
    annFile="coco2017/annotations/captions_train2017.json",
    transform=transform,
    max_samples=5000  # Ограничиваем датасет 1000 примерами
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn
)


loading annotations into memory...
Done (t=0.65s)
creating index...
index created!


In [21]:
# Проверка работы DataLoader
for images, captions in test_dataloader:
    print(f"Размер батча изображений: {images.shape}")  # Ожидается [64, 3, 224, 224]
    print(f"Количество текстовых описаний: {len(captions)}")  # Ожидается 64
    break

Размер батча изображений: torch.Size([64, 3, 224, 224])
Количество текстовых описаний: 64


In [22]:
# for images, captions in test_dataloader:
#     if torch.isnan(images).any() or torch.isinf(images).any():
#         print("Обнаружены некорректные значения в изображениях.")
#     if isinstance(captions, torch.Tensor) and (torch.isnan(captions).any() or torch.isinf(captions).any()):
#         print("Обнаружены некорректные значения в текстах.")

In [23]:
# Обучение модели
epochs = 5  # Количество эпох
best_loss = np.inf  # Лучший loss

for epoch in range(epochs):
    epoch_loss = 0.0  # Для накопления loss за эпоху
    model.train()  # Переводим модель в режим обучения

    with tqdm(
        enumerate(test_dataloader, 0),
        total=len(test_dataloader),
        desc=f"Epoch [{epoch+1}/{epochs}]",
    ) as tepoch:
        for i, (images, captions) in tepoch:
            images = images.to(device)
            captions = captions.to(device)

            # Обнуляем градиенты
            optimizer.zero_grad()

            # Получаем эмбеддинги изображений и текста
            image_features = model.encode_image(images)
            text_features = model.encode_text(captions)

            # Вычисляем loss (контрастный loss)
            logits_per_image = (image_features @ text_features.T) * model.logit_scale.exp()
            logits_per_text = logits_per_image.t()

            # Создаем метки (диагональ — правильные пары)
            labels = torch.arange(len(images), device=device)

            # Вычисляем loss для изображений и текста
            loss_image = torch.nn.functional.cross_entropy(logits_per_image, labels)
            loss_text = torch.nn.functional.cross_entropy(logits_per_text, labels)
            loss = (loss_image + loss_text) / 2

            # Сохраняем веса до обновления
            # weights_before = [param.clone() for param in model.parameters()]

            # Обратное распространение и обновление весов
            loss.backward()
            optimizer.step()

            # Проверка обновления весов
            # weights_after = list(model.parameters())
            # updated = any(not torch.allclose(before, after) for before, after in zip(weights_before, weights_after))
            # print("Весы обновлены:", updated)

            # Проверка градиентов
            # has_gradients = any(param.grad is not None and torch.any(param.grad != 0) for param in model.parameters())
            # print("Градиенты вычислены и не равны нулю:", has_gradients)

            # Обновляем progress bar
            tepoch.set_postfix(loss=loss.item())
            epoch_loss += loss.item()

    # Средний loss за эпоху
    avg_loss = epoch_loss / len(test_dataloader)

    # Вывод итогового результата эпохи
    print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.3f}")
    
    # Сохраняем модель, если loss улучшился
    if avg_loss <= best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), "clip.pt")
        print("Model Saved.")

Epoch [1/5]: 100%|██████████| 79/79 [03:57<00:00,  3.00s/it, loss=21.2]


Epoch [1/5], Average Loss: 73.038
Model Saved.


Epoch [2/5]: 100%|██████████| 79/79 [03:56<00:00,  3.00s/it, loss=7.21]


Epoch [2/5], Average Loss: 31.254
Model Saved.


Epoch [3/5]: 100%|██████████| 79/79 [03:56<00:00,  2.99s/it, loss=0]   


Epoch [3/5], Average Loss: 20.634
Model Saved.


Epoch [4/5]: 100%|██████████| 79/79 [03:56<00:00,  3.00s/it, loss=8.01]


Epoch [4/5], Average Loss: 15.526
Model Saved.


Epoch [5/5]: 100%|██████████| 79/79 [03:55<00:00,  2.98s/it, loss=3.68]


Epoch [5/5], Average Loss: 11.813
Model Saved.
